In [1]:
from gurobipy import *
import pandas as pd
import numpy as np

In [2]:
# create a new model
myModel = Model("Final_Project")

# create decision variables and integrate them into the model
k_s = 3
i_s = 5

# vars storage
x_s = [[0 for i in range(i_s)] for k in range(k_s)]
y_s = [[[0 for i in range(i_s)] for k in range(k_s)] for k in range(k_s)]
n_s = [[[0 for i in range(i_s)] for k in range(k_s)] for k in range(k_s)]
m_s = [[[0 for i in range(i_s)] for k in range(k_s)] for k in range(k_s)]
z_s = [[[0 for i in range(i_s)] for k in range(k_s)] for k in range(k_s)]

# z_s (zk1k2i) #cargo tasks to K1-K2 path.
z_s[0][1]=[100,200,100,400,300]
z_s[0][2]=[50,50,50,50,50]
z_s[1][0]=[25,25,25,25,25]
z_s[1][2]=[25,25,25,25,25]
z_s[2][0]=[40,40,40,40,40]
z_s[2][1]=[400,200,300,200,400]

Academic license - for non-commercial use only


In [3]:
# x_s (xki) avaliable #aircraft at airport K at day i (morning)
for k in range(k_s):
    for i in range(i_s):
        xVar = myModel.addVar(vtype=GRB.INTEGER, name='x' + str(k+1) + ',' + str(i+1))
        x_s[k][i] = xVar
        
myModel.update()

# y_s (yk1k2i) #aircraft transfer from K1 to K2 in day i
for k1 in range(k_s):
    for k2 in range(k_s):
        if k1 != k2:
            for i in range(i_s):
                yVar = myModel.addVar(vtype=GRB.INTEGER, name='y' + str(k1+1) + ',' + str(k2+1) + ',' + str(i+1))
                y_s[k1][k2][i] = yVar
        
myModel.update()

# n_s (nk1k2i) #cargo tasks left from K1 to K2 at day i (morning) before adding new tasks
for k1 in range(k_s):
    for k2 in range(k_s):
        if k1 != k2:
            for i in range(i_s):
                nVar = myModel.addVar(vtype=GRB.INTEGER, name='n' + str(k1+1) + ',' + str(k2+1) + ',' + str(i+1))
                n_s[k1][k2][i] = nVar
        
myModel.update()

# m_s (mk1k2i) #cargo tasks left from K1 to K2 at day i (morning) before adding new tasks
for k1 in range(k_s):
    for k2 in range(k_s):
        if k1 != k2:
            for i in range(i_s):
                mVar = myModel.addVar(vtype=GRB.INTEGER, name='m' + str(k1+1) + ',' + str(k2+1) + ',' + str(i+1))
                m_s[k1][k2][i] = mVar
        
myModel.update()


# create a linear expression for the objective
objExpr = LinExpr()
for k1 in range(k_s):
    for k2 in range(k_s):
        if k1 != k2:
            for i in range(i_s):
                nVar = n_s[k1][k2][i]
                objExpr += 10*nVar
                myModel.setObjective(objExpr, GRB.MINIMIZE)

myModel.update()

# create a linear expression for the objective
objExpr = LinExpr()
for k1 in range(k_s):
    for k2 in range(k_s):
        if k1 != k2:
            nVar = n_s[k1][k2][0]
            objExpr += 30*nVar
            for i in range(1,i_s):
                nVar = n_s[k1][k2][i]
                objExpr += 10*nVar
            myModel.setObjective(objExpr, GRB.MINIMIZE)
for i in range(i_s):
    yVar1 = y_s[0][1][i]
    mVar1 = m_s[0][1][i]
    yVar2 = y_s[1][0][i]
    mVar2 = m_s[1][0][i]
    objExpr += 7*(yVar1-mVar1+yVar2-mVar2)
    myModel.setObjective(objExpr, GRB.MINIMIZE)
for i in range(i_s):
    yVar1 = y_s[0][2][i]
    mVar1 = m_s[0][2][i]
    yVar2 = y_s[2][0][i]
    mVar2 = m_s[2][0][i]
    objExpr += 3*(yVar1-mVar1+yVar2-mVar2)
    myModel.setObjective(objExpr, GRB.MINIMIZE)
for i in range(i_s):
    yVar1 = y_s[1][2][i]
    mVar1 = m_s[1][2][i]
    yVar2 = y_s[2][1][i]
    mVar2 = m_s[2][1][i]
    objExpr += 6*(yVar1-mVar1+yVar2-mVar2)
    myModel.setObjective(objExpr, GRB.MINIMIZE)
                
myModel.update()

# Type 1 constraint: Constraint for number of aircraft on the first day
constExpr = LinExpr()
for k in range(k_s):
    xVar = x_s[k][0]
    constExpr += xVar
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 1200, name="total_aircraft")

# Type 2 constraint: Constraint for flight balance
# 5-1-A
constExpr = LinExpr()
constExpr += x_s[0][0] - x_s[0][4]+y_s[0][1][4]+y_s[0][2][4]-y_s[1][0][4]-y_s[2][0][4]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_5_1_A")
# 5-1-B
constExpr = LinExpr()
constExpr += x_s[1][0] - x_s[1][4]+y_s[1][0][4]+y_s[1][2][4]-y_s[0][1][4]-y_s[2][1][4]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_5_1_B")
# 5-1-C
constExpr = LinExpr()
constExpr += x_s[2][0] - x_s[2][4]+y_s[2][0][4]+y_s[2][1][4]-y_s[0][2][4]-y_s[1][2][4]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_5_1_C")

# 2-1-A
constExpr = LinExpr()
constExpr += x_s[0][1] - x_s[0][0]+y_s[0][1][0]+y_s[0][2][0]-y_s[1][0][0]-y_s[2][0][0]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_2_1_A")
# 2-1-B
constExpr = LinExpr()
constExpr += x_s[1][1] - x_s[1][0]+y_s[1][0][0]+y_s[1][2][0]-y_s[0][1][0]-y_s[2][1][0]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_2_1_B")
# 2-1-C
constExpr = LinExpr()
constExpr += x_s[2][1] - x_s[2][0]+y_s[2][0][0]+y_s[2][1][0]-y_s[0][2][0]-y_s[1][2][0]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_2_1_C")

# 3-2-A
constExpr = LinExpr()
constExpr += x_s[0][2] - x_s[0][1]+y_s[0][1][1]+y_s[0][2][1]-y_s[1][0][1]-y_s[2][0][1]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_3_2_A")
# 3-2-B
constExpr = LinExpr()
constExpr += x_s[1][2] - x_s[1][1]+y_s[1][0][1]+y_s[1][2][1]-y_s[0][1][1]-y_s[2][1][1]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_3_2_B")
# 3-2-C
constExpr = LinExpr()
constExpr += x_s[2][2] - x_s[2][1]+y_s[2][0][1]+y_s[2][1][1]-y_s[0][2][1]-y_s[1][2][1]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_3_2_C")

# 4-3-A
constExpr = LinExpr()
constExpr += x_s[0][3] - x_s[0][2]+y_s[0][1][2]+y_s[0][2][2]-y_s[1][0][2]-y_s[2][0][2]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_4_3_A")
# 4-3-B
constExpr = LinExpr()
constExpr += x_s[1][3] - x_s[1][2]+y_s[1][0][2]+y_s[1][2][2]-y_s[0][1][2]-y_s[2][1][2]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_4_3_B")
# 4-3-C
constExpr = LinExpr()
constExpr += x_s[2][3] - x_s[2][2]+y_s[2][0][2]+y_s[2][1][2]-y_s[0][2][2]-y_s[1][2][2]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_4_3_C")

# 5-4-A
constExpr = LinExpr()
constExpr += x_s[0][4] - x_s[0][3]+y_s[0][1][3]+y_s[0][2][3]-y_s[1][0][3]-y_s[2][0][3]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_5_4_A")
# 5-4-B
constExpr = LinExpr()
constExpr += x_s[1][4] - x_s[1][3]+y_s[1][0][3]+y_s[1][2][3]-y_s[0][1][3]-y_s[2][1][3]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_5_4_B")
# 5-4-C
constExpr = LinExpr()
constExpr += x_s[2][4] - x_s[2][3]+y_s[2][0][3]+y_s[2][1][3]-y_s[0][2][3]-y_s[1][2][3]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="aircraft_5_4_C")

# Type 3 constraint: #avaliable aircraft >= #aircraft out
for i in range(i_s):
    constExpr = LinExpr()
    constExpr += x_s[0][i] - y_s[0][1][i] - y_s[0][2][i]
    myModel.addConstr(lhs = constExpr, sense = GRB.GREATER_EQUAL, rhs = 0, name="airport_A_aircraft_requirement_"+str(i+1))
    constExpr = LinExpr()
    constExpr += x_s[1][i] - y_s[1][0][i] - y_s[1][2][i]
    myModel.addConstr(lhs = constExpr, sense = GRB.GREATER_EQUAL, rhs = 0, name="airport_B_aircraft_requirement_"+str(i+1))
    constExpr = LinExpr()
    constExpr += x_s[2][i] - y_s[2][0][i] - y_s[2][1][i]
    myModel.addConstr(lhs = constExpr, sense = GRB.GREATER_EQUAL, rhs = 0, name="airport_C_aircraft_requirement_"+str(i+1))

# Type 4 constraint: #aircraft our >= #cargo out
for k1 in range(k_s):
    for k2 in range(k_s):
        if k1 != k2:
            for i in range(i_s):
                constExpr = LinExpr()
                constExpr += y_s[k1][k2][i] - m_s[k1][k2][i]
                myModel.addConstr(lhs = constExpr, sense = GRB.GREATER_EQUAL, rhs = 0, name="aircraft_cargo_requirement_"+str(k1+1)+str(k2+1)+str(i+1))
                
# Type 5 constraint: #left cargo balance
# AB_5_1
constExpr = LinExpr()
constExpr += n_s[0][1][0] - n_s[0][1][4] - z_s[0][1][4] + m_s[0][1][4]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_AB_5_1")
# AC_5_1
constExpr = LinExpr()
constExpr += n_s[0][2][0] - n_s[0][2][4] - z_s[0][2][4] + m_s[0][2][4]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_AC_5_1")
# BA_5_1
constExpr = LinExpr()
constExpr += n_s[1][0][0] - n_s[1][0][4] - z_s[1][0][4] + m_s[1][0][4]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_BA_5_1")
# BC_5_1
constExpr = LinExpr()
constExpr += n_s[1][2][0] - n_s[1][2][4] - z_s[1][2][4] + m_s[1][2][4]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_BC_5_1")
# CA_5_1
constExpr = LinExpr()
constExpr += n_s[2][0][0] - n_s[2][0][4] - z_s[2][0][4] + m_s[2][0][4]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_CA_5_1")
# CB_5_1
constExpr = LinExpr()
constExpr += n_s[2][1][0] - n_s[2][1][4] - z_s[2][1][4] + m_s[2][1][4]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_CB_5_1")

# AB_2_1
constExpr = LinExpr()
constExpr += n_s[0][1][1] - n_s[0][1][0] - z_s[0][1][0] + m_s[0][1][0]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_AB_2_1")
# AC_2_1
constExpr = LinExpr()
constExpr += n_s[0][2][1] - n_s[0][2][0] - z_s[0][2][0] + m_s[0][2][0]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_AC_2_1")
# BA_2_1
constExpr = LinExpr()
constExpr += n_s[1][0][1] - n_s[1][0][0] - z_s[1][0][0] + m_s[1][0][0]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_BA_2_1")
# BC_2_1
constExpr = LinExpr()
constExpr += n_s[1][2][1] - n_s[1][2][0] - z_s[1][2][0] + m_s[1][2][0]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_BC_2_1")
# CA_2_1
constExpr = LinExpr()
constExpr += n_s[2][0][1] - n_s[2][0][0] - z_s[2][0][0] + m_s[2][0][0]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_CA_2_1")
# CB_2_1
constExpr = LinExpr()
constExpr += n_s[2][1][1] - n_s[2][1][0] - z_s[2][1][0] + m_s[2][1][0]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_CB_2_1")

# AB_3_2
constExpr = LinExpr()
constExpr += n_s[0][1][2] - n_s[0][1][1] - z_s[0][1][1] + m_s[0][1][1]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_AB_3_2")
# AC_3_2
constExpr = LinExpr()
constExpr += n_s[0][2][2] - n_s[0][2][1] - z_s[0][2][1] + m_s[0][2][1]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_AC_3_2")
# BA_3_2
constExpr = LinExpr()
constExpr += n_s[1][0][2] - n_s[1][0][1] - z_s[1][0][1] + m_s[1][0][1]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_BA_3_2")
# BC_3_2
constExpr = LinExpr()
constExpr += n_s[1][2][2] - n_s[1][2][1] - z_s[1][2][1] + m_s[1][2][1]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_BC_3_2")
# CA_3_2
constExpr = LinExpr()
constExpr += n_s[2][0][2] - n_s[2][0][1] - z_s[2][0][1] + m_s[2][0][1]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_CA_3_2")
# CB_3_2
constExpr = LinExpr()
constExpr += n_s[2][1][2] - n_s[2][1][1] - z_s[2][1][1] + m_s[2][1][1]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_CB_3_2")

# AB_4_3
constExpr = LinExpr()
constExpr += n_s[0][1][3] - n_s[0][1][2] - z_s[0][1][2] + m_s[0][1][2]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_AB_4_3")
# AC_4_3
constExpr = LinExpr()
constExpr += n_s[0][2][3] - n_s[0][2][2] - z_s[0][2][2] + m_s[0][2][2]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_AC_4_3")
# BA_4_3
constExpr = LinExpr()
constExpr += n_s[1][0][3] - n_s[1][0][2] - z_s[1][0][2] + m_s[1][0][2]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_BA_4_3")
# BC_4_3
constExpr = LinExpr()
constExpr += n_s[1][2][3] - n_s[1][2][2] - z_s[1][2][2] + m_s[1][2][2]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_BC_4_3")
# CA_4_3
constExpr = LinExpr()
constExpr += n_s[2][0][3] - n_s[2][0][2] - z_s[2][0][2] + m_s[2][0][2]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_CA_4_3")
# CB_4_3
constExpr = LinExpr()
constExpr += n_s[2][1][3] - n_s[2][1][2] - z_s[2][1][2] + m_s[2][1][2]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_CB_4_3")

# AB_5_4
constExpr = LinExpr()
constExpr += n_s[0][1][4] - n_s[0][1][3] - z_s[0][1][3] + m_s[0][1][3]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_AB_5_4")
# AC_5_4
constExpr = LinExpr()
constExpr += n_s[0][2][4] - n_s[0][2][3] - z_s[0][2][3] + m_s[0][2][3]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_AC_5_4")
# BA_5_4
constExpr = LinExpr()
constExpr += n_s[1][0][4] - n_s[1][0][3] - z_s[1][0][3] + m_s[1][0][3]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_BA_5_4")
# BC_5_4
constExpr = LinExpr()
constExpr += n_s[1][2][4] - n_s[1][2][3] - z_s[1][2][3] + m_s[1][2][3]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_BC_5_4")
# CA_5_4
constExpr = LinExpr()
constExpr += n_s[2][0][4] - n_s[2][0][3] - z_s[2][0][3] + m_s[2][0][3]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_CA_5_4")
# CB_5_4
constExpr = LinExpr()
constExpr += n_s[2][1][4] - n_s[2][1][3] - z_s[2][1][3] + m_s[2][1][3]
myModel.addConstr(lhs = constExpr, sense = GRB.EQUAL, rhs = 0, name="left_cargo_balance_CB_5_4")

###############################################################################

# integrate objective and constraints into the model
myModel.update()

# write the model in a file to make sure it is constructed correctly
myModel.write(filename="Project.lp")

# optimize the model
myModel.optimize()

# save result to xls file
allVars = myModel.getVars()

# writebook = xlwt.Workbook()
# sheet1 = writebook.add_sheet('test')

# for i in range(71):
#     for j in range(21):
#         if(allVars[21*i+j].x == 1):
#             sheet1.write(i+1,j+1,sheet.cell(i+1,j+1).value)

# writebook.save('result.xls')

Optimize a model with 91 rows, 105 columns and 288 nonzeros
Variable types: 0 continuous, 105 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+03]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 90 rows, 104 columns, 288 nonzeros
Variable types: 0 continuous, 104 integer (0 binary)

Root relaxation: objective 2.172500e+04, 71 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    21725.000000 21725.0000  0.00%     -    0s

Explored 0 nodes (71 simplex iterations) in 0.11 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 21725 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.172500000000e+04, best bound 2.172500000000e+04, gap 0.0000%


In [4]:
x_s

[[<gurobi.Var x1,1 (value 65.0)>,
  <gurobi.Var x1,2 (value 335.0)>,
  <gurobi.Var x1,3 (value 150.0)>,
  <gurobi.Var x1,4 (value 450.0)>,
  <gurobi.Var x1,5 (value 350.0)>],
 [<gurobi.Var x2,1 (value 510.0)>,
  <gurobi.Var x2,2 (value 620.0)>,
  <gurobi.Var x2,3 (value 485.0)>,
  <gurobi.Var x2,4 (value 400.0)>,
  <gurobi.Var x2,5 (value 600.0)>],
 [<gurobi.Var x3,1 (value 625.0)>,
  <gurobi.Var x3,2 (value 245.0)>,
  <gurobi.Var x3,3 (value 565.0)>,
  <gurobi.Var x3,4 (value 350.0)>,
  <gurobi.Var x3,5 (value 250.0)>]]

In [5]:
y_s

[[[0, 0, 0, 0, 0],
  [<gurobi.Var y1,2,1 (value 15.0)>,
   <gurobi.Var y1,2,2 (value 285.0)>,
   <gurobi.Var y1,2,3 (value 100.0)>,
   <gurobi.Var y1,2,4 (value 400.0)>,
   <gurobi.Var y1,2,5 (value 300.0)>],
  [<gurobi.Var y1,3,1 (value 50.0)>,
   <gurobi.Var y1,3,2 (value 50.0)>,
   <gurobi.Var y1,3,3 (value 50.0)>,
   <gurobi.Var y1,3,4 (value 50.0)>,
   <gurobi.Var y1,3,5 (value 50.0)>]],
 [[<gurobi.Var y2,1,1 (value 300.0)>,
   <gurobi.Var y2,1,2 (value 105.0)>,
   <gurobi.Var y2,1,3 (value 410.0)>,
   <gurobi.Var y2,1,4 (value 310.0)>,
   <gurobi.Var y2,1,5 (value 25.0)>],
  [0, 0, 0, 0, 0],
  [<gurobi.Var y2,3,1 (value 195.0)>,
   <gurobi.Var y2,3,2 (value 515.0)>,
   <gurobi.Var y2,3,3 (value 75.0)>,
   <gurobi.Var y2,3,4 (value 90.0)>,
   <gurobi.Var y2,3,5 (value 575.0)>]],
 [[<gurobi.Var y3,1,1 (value 35.0)>,
   <gurobi.Var y3,1,2 (value 45.0)>,
   <gurobi.Var y3,1,3 (value 40.0)>,
   <gurobi.Var y3,1,4 (value 40.0)>,
   <gurobi.Var y3,1,5 (value 40.0)>],
  [<gurobi.Var y3,2

In [6]:
n_s

[[[0, 0, 0, 0, 0],
  [<gurobi.Var n1,2,1 (value -0.0)>,
   <gurobi.Var n1,2,2 (value 85.0)>,
   <gurobi.Var n1,2,3 (value -0.0)>,
   <gurobi.Var n1,2,4 (value -0.0)>,
   <gurobi.Var n1,2,5 (value -0.0)>],
  [<gurobi.Var n1,3,1 (value -0.0)>,
   <gurobi.Var n1,3,2 (value -0.0)>,
   <gurobi.Var n1,3,3 (value -0.0)>,
   <gurobi.Var n1,3,4 (value -0.0)>,
   <gurobi.Var n1,3,5 (value -0.0)>]],
 [[<gurobi.Var n2,1,1 (value -0.0)>,
   <gurobi.Var n2,1,2 (value -0.0)>,
   <gurobi.Var n2,1,3 (value -0.0)>,
   <gurobi.Var n2,1,4 (value -0.0)>,
   <gurobi.Var n2,1,5 (value -0.0)>],
  [0, 0, 0, 0, 0],
  [<gurobi.Var n2,3,1 (value -0.0)>,
   <gurobi.Var n2,3,2 (value -0.0)>,
   <gurobi.Var n2,3,3 (value -0.0)>,
   <gurobi.Var n2,3,4 (value -0.0)>,
   <gurobi.Var n2,3,5 (value -0.0)>]],
 [[<gurobi.Var n3,1,1 (value -0.0)>,
   <gurobi.Var n3,1,2 (value 5.0)>,
   <gurobi.Var n3,1,3 (value -0.0)>,
   <gurobi.Var n3,1,4 (value -0.0)>,
   <gurobi.Var n3,1,5 (value -0.0)>],
  [<gurobi.Var n3,2,1 (value 19

In [7]:
m_s

[[[0, 0, 0, 0, 0],
  [<gurobi.Var m1,2,1 (value 15.0)>,
   <gurobi.Var m1,2,2 (value 285.0)>,
   <gurobi.Var m1,2,3 (value 100.0)>,
   <gurobi.Var m1,2,4 (value 400.0)>,
   <gurobi.Var m1,2,5 (value 300.0)>],
  [<gurobi.Var m1,3,1 (value 50.0)>,
   <gurobi.Var m1,3,2 (value 50.0)>,
   <gurobi.Var m1,3,3 (value 50.0)>,
   <gurobi.Var m1,3,4 (value 50.0)>,
   <gurobi.Var m1,3,5 (value 50.0)>]],
 [[<gurobi.Var m2,1,1 (value 25.0)>,
   <gurobi.Var m2,1,2 (value 25.0)>,
   <gurobi.Var m2,1,3 (value 25.0)>,
   <gurobi.Var m2,1,4 (value 25.0)>,
   <gurobi.Var m2,1,5 (value 25.0)>],
  [0, 0, 0, 0, 0],
  [<gurobi.Var m2,3,1 (value 25.0)>,
   <gurobi.Var m2,3,2 (value 25.0)>,
   <gurobi.Var m2,3,3 (value 25.0)>,
   <gurobi.Var m2,3,4 (value 25.0)>,
   <gurobi.Var m2,3,5 (value 25.0)>]],
 [[<gurobi.Var m3,1,1 (value 35.0)>,
   <gurobi.Var m3,1,2 (value 45.0)>,
   <gurobi.Var m3,1,3 (value 40.0)>,
   <gurobi.Var m3,1,4 (value 40.0)>,
   <gurobi.Var m3,1,5 (value 40.0)>],
  [<gurobi.Var m3,2,1 (val